# 6407 Симовин Кирилл. Вариант 3. Лабораторная работа 4

Tensorflow 2.x

1) Подготовка данных

2) Использование Keras Model API

3) Использование Keras Sequential + Functional API

https://www.tensorflow.org/tutorials

Для выполнения лабораторной работы необходимо установить tensorflow версии 2.0 или выше .

Рекомендуется использовать возможности Colab'а по обучению моделей на GPU.



In [1]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
USE_GPU = False
device = '/device:GPU:0' if USE_GPU else '/cpu:0'

In [3]:
print_every = 100

# Подготовка данных
Загрузите набор данных из предыдущей лабораторной работы.

In [4]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

170498071/170498071 [==============================] - 7s 0us/step
Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [5]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y

        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [6]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 32, 32, 3) (64,)
1 (64, 32, 32, 3) (64,)
2 (64, 32, 32, 3) (64,)
3 (64, 32, 32, 3) (64,)
4 (64, 32, 32, 3) (64,)
5 (64, 32, 32, 3) (64,)
6 (64, 32, 32, 3) (64,)


#  Keras Model Subclassing API


Для реализации собственной модели с помощью Keras Model Subclassing API необходимо выполнить следующие шаги:

1) Определить новый класс, который является наследником tf.keras.Model.

2) В методе __init__() определить все необходимые слои из модуля tf.keras.layer

3) Реализовать прямой проход в методе call() на основе слоев, объявленных в __init__()

Ниже приведен пример использования keras API для определения двухслойной полносвязной сети.

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras

In [7]:
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()

    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    with tf.device(device):
        scores = model(x)
        print(scores.shape)

test_TwoLayerFC()

(64, 10)


/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Реализуйте трехслойную CNN для вашей задачи классификации.

Архитектура сети:
    
1. Сверточный слой (5 x 5 kernels, zero-padding = 'same')
2. Функция активации ReLU
3. Сверточный слой (3 x 3 kernels, zero-padding = 'same')
4. Функция активации ReLU
5. Полносвязный слой
6. Функция активации Softmax

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [8]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # TODO: Implement the __init__ method for a three-layer ConvNet. You   #
        # should instantiate layer objects to be used in the forward pass.     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        self.conv1 = tf.keras.layers.Conv2D(channel_1, (5, 5), padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(channel_2, (3, 3), padding='same', activation='relu')
        self.flatten = tf.keras.layers.Flatten()
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################

    def call(self, x, training=False):
        scores = None
        ########################################################################
        # TODO: Implement the forward pass for a three-layer ConvNet. You      #
        # should use the layer objects defined in the __init__ method.         #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        scores = self.conv1(x)
        scores = self.conv2(scores)
        scores = self.flatten(scores)
        scores = self.fc(scores)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################
        return scores

In [9]:
def test_ThreeLayerConvNet():
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 3, 32, 32))
        scores = model(x)
        print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


Пример реализации процесса обучения:

In [10]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.

    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for

    Returns: Nothing, but prints progress during trainingn
    """
    with tf.device(device):


        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

        model = model_init_fn()
        optimizer = optimizer_init_fn()

        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

        t = 0
        for epoch in range(num_epochs):

            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_states()
            train_accuracy.reset_states()

            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:

                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)

                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)

                    if t % print_every == 0:
                        val_loss.reset_states()
                        val_accuracy.reset_states()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)

                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [11]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.919250726699829, Accuracy: 12.5, Val Loss: 2.916606903076172, Val Accuracy: 14.100000381469727
Iteration 100, Epoch 1, Loss: 2.236943483352661, Accuracy: 28.991334915161133, Val Loss: 1.9026331901550293, Val Accuracy: 35.900001525878906
Iteration 200, Epoch 1, Loss: 2.0773301124572754, Accuracy: 32.15174102783203, Val Loss: 1.8273539543151855, Val Accuracy: 40.0
Iteration 300, Epoch 1, Loss: 1.9995861053466797, Accuracy: 33.98048400878906, Val Loss: 1.889496088027954, Val Accuracy: 37.099998474121094
Iteration 400, Epoch 1, Loss: 1.9297997951507568, Accuracy: 35.816707611083984, Val Loss: 1.7173644304275513, Val Accuracy: 42.20000076293945
Iteration 500, Epoch 1, Loss: 1.88694167137146, Accuracy: 36.84817886352539, Val Loss: 1.661793828010559, Val Accuracy: 43.0
Iteration 600, Epoch 1, Loss: 1.8582065105438232, Accuracy: 37.754783630371094, Val Loss: 1.685495138168335, Val Accuracy: 42.39999771118164
Iteration 700, Epoch 1, Loss: 1.8337353467941284, Accura

Обучите трехслойную CNN. В tf.keras.optimizers.SGD укажите Nesterov momentum = 0.9 .

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD

Значение accuracy на валидационной выборке после 1 эпохи обучения должно быть > 50% .

In [12]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.3047313690185547, Accuracy: 12.5, Val Loss: 2.3415966033935547, Val Accuracy: 9.600000381469727
Iteration 100, Epoch 1, Loss: 1.9277524948120117, Accuracy: 30.67759895324707, Val Loss: 1.6836179494857788, Val Accuracy: 42.39999771118164
Iteration 200, Epoch 1, Loss: 1.7720316648483276, Accuracy: 37.0335807800293, Val Loss: 1.488687515258789, Val Accuracy: 47.20000076293945
Iteration 300, Epoch 1, Loss: 1.6775693893432617, Accuracy: 40.422550201416016, Val Loss: 1.460410475730896, Val Accuracy: 47.400001525878906
Iteration 400, Epoch 1, Loss: 1.6064118146896362, Accuracy: 42.89276885986328, Val Loss: 1.3586362600326538, Val Accuracy: 52.39999771118164
Iteration 500, Epoch 1, Loss: 1.5571049451828003, Accuracy: 44.62948989868164, Val Loss: 1.3199578523635864, Val Accuracy: 55.0
Iteration 600, Epoch 1, Loss: 1.5268827676773071, Accuracy: 45.70767593383789, Val Loss: 1.3069486618041992, Val Accuracy: 54.10000228881836
Iteration 700, Epoch 1, Loss: 1.4980040788

# Использование Keras Sequential API для реализации последовательных моделей.

Пример для полносвязной сети:

In [13]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (32, 32, 3)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax',
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.7565040588378906, Accuracy: 17.1875, Val Loss: 2.9342777729034424, Val Accuracy: 12.600000381469727
Iteration 100, Epoch 1, Loss: 2.2724602222442627, Accuracy: 27.4907169342041, Val Loss: 1.8812997341156006, Val Accuracy: 37.20000076293945
Iteration 200, Epoch 1, Loss: 2.0980749130249023, Accuracy: 31.786378860473633, Val Loss: 1.8550039529800415, Val Accuracy: 38.5
Iteration 300, Epoch 1, Loss: 2.015765428543091, Accuracy: 33.88704299926758, Val Loss: 1.8749051094055176, Val Accuracy: 38.20000076293945
Iteration 400, Epoch 1, Loss: 1.9437768459320068, Accuracy: 35.79722595214844, Val Loss: 1.72527015209198, Val Accuracy: 41.20000076293945
Iteration 500, Epoch 1, Loss: 1.8978321552276611, Accuracy: 36.85129928588867, Val Loss: 1.6820099353790283, Val Accuracy: 41.80000305175781
Iteration 600, Epoch 1, Loss: 1.8660513162612915, Accuracy: 37.73658752441406, Val Loss: 1.7201509475708008, Val Accuracy: 40.5
Iteration 700, Epoch 1, Loss: 1.839095115661621, Accu

Альтернативный менее гибкий способ обучения:

In [14]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

313/313 [==============================] - 9s 27ms/step - loss: 1.8885 - sparse_categorical_accuracy: 0.3852


[1.8884981870651245, 0.38519999384880066]

Перепишите реализацию трехслойной CNN с помощью tf.keras.Sequential API . Обучите модель двумя способами.

In [15]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = tf.keras.Sequential([
        # Первый сверточный слой
        tf.keras.layers.Conv2D(32, (5, 5), padding='same', activation='relu', input_shape=(32, 32, 3)),
        # Первый пулинг слой
        tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),
        # Второй сверточный слой
        tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
        # Второй пулинг слой
        tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),
        # Разворачиваем данные перед подачей на полносвязный слой
        tf.keras.layers.Flatten(),
        # Полносвязный слой с relu активацией
        tf.keras.layers.Dense(1024, activation='relu'),
        # Выходной слой с softmax активацией
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.400045394897461, Accuracy: 7.8125, Val Loss: 2.3487019538879395, Val Accuracy: 9.800000190734863
Iteration 100, Epoch 1, Loss: 2.3121964931488037, Accuracy: 10.179455757141113, Val Loss: 2.2647173404693604, Val Accuracy: 16.799999237060547
Iteration 200, Epoch 1, Loss: 2.2858731746673584, Accuracy: 13.230721473693848, Val Loss: 2.2296252250671387, Val Accuracy: 20.5
Iteration 300, Epoch 1, Loss: 2.2674787044525146, Accuracy: 15.043603897094727, Val Loss: 2.201352596282959, Val Accuracy: 23.100000381469727
Iteration 400, Epoch 1, Loss: 2.2472667694091797, Accuracy: 17.039432525634766, Val Loss: 2.168119192123413, Val Accuracy: 24.299999237060547
Iteration 500, Epoch 1, Loss: 2.2315919399261475, Accuracy: 18.503618240356445, Val Loss: 2.1408586502075195, Val Accuracy: 25.200000762939453
Iteration 600, Epoch 1, Loss: 2.2157421112060547, Accuracy: 19.79253387451172, Val Loss: 2.114121198654175, Val Accuracy: 26.69999885559082
Iteration 700, Epoch 1, Loss: 2.19

In [16]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

313/313 [==============================] - 13s 41ms/step - loss: 1.4350 - sparse_categorical_accuracy: 0.4971


[1.435001254081726, 0.49709999561309814]

# Использование Keras Functional API

Для реализации более сложных архитектур сети с несколькими входами/выходами, повторным использованием слоев, "остаточными" связями (residual connections) необходимо явно указать входные и выходные тензоры.

Ниже представлен пример для полносвязной сети.

In [ ]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)

    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)

    with tf.device(device):
        scores = model(x)
        print(scores.shape)

test_two_layer_fc_functional()

(64, 10)


In [ ]:
input_shape = (32, 32, 3)
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.8214259147644043, Accuracy: 9.375, Val Loss: 2.6591882705688477, Val Accuracy: 15.5
Iteration 100, Epoch 1, Loss: 2.215625762939453, Accuracy: 28.790224075317383, Val Loss: 1.9095675945281982, Val Accuracy: 36.5
Iteration 200, Epoch 1, Loss: 2.065948247909546, Accuracy: 32.462684631347656, Val Loss: 1.8642170429229736, Val Accuracy: 39.89999771118164
Iteration 300, Epoch 1, Loss: 2.000307321548462, Accuracy: 34.099876403808594, Val Loss: 1.8720731735229492, Val Accuracy: 38.5
Iteration 400, Epoch 1, Loss: 1.9299927949905396, Accuracy: 35.960880279541016, Val Loss: 1.756574273109436, Val Accuracy: 41.29999923706055
Iteration 500, Epoch 1, Loss: 1.8892277479171753, Accuracy: 36.96669006347656, Val Loss: 1.6759389638900757, Val Accuracy: 42.599998474121094
Iteration 600, Epoch 1, Loss: 1.8584928512573242, Accuracy: 37.92897033691406, Val Loss: 1.7090699672698975, Val Accuracy: 42.79999923706055
Iteration 700, Epoch 1, Loss: 1.8323469161987305, Accuracy: 38.64

Поэкспериментируйте с архитектурой сверточной сети. Для вашего набора данных вам необходимо получить как минимум 70% accuracy на валидационной выборке за 10 эпох обучения. Опишите все эксперименты и сделайте выводы (без выполнения данного пункта работы приниматься не будут).

Эспериментируйте с архитектурой, гиперпараметрами, функцией потерь, регуляризацией, методом оптимизации.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

In [ ]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        # Дробавляем входной сверточный слой с функцией активации 'relu' и 16 фильтрами
        self.conv1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(32, 32, 3))

        # Добавим скрытый сверточный слой с ядром 3 x 3 и функцией активации 'relu'
        self.conv2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')

        # Максимальный пулинг 2 х 2
        self.pool1 = tf.keras.layers.MaxPooling2D((2, 2))

        # Добавляем дропаут в 0.25
        self.dropout1 = tf.keras.layers.Dropout(0.25)

        # Добавим скрытый сверточный слой с ядром 3 x 3 и функцией активации 'relu'
        self.conv3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')

        # Максимальный пулинг 2 х 2
        self.pool2 = tf.keras.layers.MaxPooling2D((2, 2))

        # Слой для развертывания
        self.flatten = tf.keras.layers.Flatten()

        # Полносвязный слой с функцией активации 'relu'
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')

        # Добавляем дропаут в 0.4
        self.dropout2 = tf.keras.layers.Dropout(0.4)

        # Выходной полносвязный слой с сигмоидой в качестве функции активации
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################

    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(input_tensor)
        x = self.conv2(x)
        x = self.pool1(x)

        if training:
            x = self.dropout1(x, training=training)

        x = self.conv3(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.dense1(x)

        if training:
            x = self.dropout2(x, training=training)

        x = self.dense2(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x


print_every = 700
num_epochs = 4

model = CustomConvNet()

def model_init_fn():
    return CustomConvNet()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate)

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 2.3869314193725586, Accuracy: 10.9375, Val Loss: 2.3323779106140137, Val Accuracy: 14.5
Iteration 700, Epoch 1, Loss: 1.5740876197814941, Accuracy: 42.85841751098633, Val Loss: 1.1937273740768433, Val Accuracy: 58.70000076293945
Iteration 1400, Epoch 2, Loss: 1.20307457447052, Accuracy: 57.16535568237305, Val Loss: 0.9855291247367859, Val Accuracy: 66.29999542236328
Iteration 2100, Epoch 3, Loss: 1.0471820831298828, Accuracy: 63.38422775268555, Val Loss: 0.9173915982246399, Val Accuracy: 69.0999984741211
Iteration 2800, Epoch 4, Loss: 0.946917712688446, Accuracy: 67.05392456054688, Val Loss: 0.8573281168937683, Val Accuracy: 71.1000015258789


Опишите все эксперименты, результаты. Сделайте выводы.

1. Архитектура сети состоит из: сверточных слоев, максимального пулинга, droput, flatten и полносвязных слоев.
2. На одинаковых настройках были протестированы различные оптимизаторы: SGD, Adam, RMSprop. Первый показал наихудшие результаты - за 10 эпох точность была в районе 30%. Последние два показали примерно одинаковые результаты, но в большинстве своем Adam был немножко точнее.
3. На входном и скрытом слоях сравнивались функции активации relu и sigmoid, а на входном слое тестировались sigmoid и softmax. При комбинации sigmoid + softmax + Adam получился наихудший результат - для получения необходимой точности потребовалось в районе 13 эпох. При комбинации relu + softmax + Adam был получен наилучший результат - 4 эпохи. В связи с этим число эпох для обучения было уменьшен с 10 до 4.